## Log in

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from koapy import KiwoomOpenApiPlusEntrypoint
from koapy import KiwoomOpenApiPlusTrInfo

In [2]:
entrypoint = KiwoomOpenApiPlusEntrypoint()
entrypoint.EnsureConnected()
print(entrypoint.GetConnectState())

2024-02-17 11:00:23,009 [DEBUG] Testing if client is ready... - KiwoomOpenApiPlusEntrypoint.py:45
2024-02-17 11:00:33,090 [DEBUG] Client is not ready - KiwoomOpenApiPlusEntrypoint.py:47
2024-02-17 11:00:33,093 [DEBUG] Creating a new server... - KiwoomOpenApiPlusEntrypoint.py:48


1


## Code

In [77]:
def call_TR(trcode:str, param:list, screenno=None):
    output = {'single': {}, 'multi': []}
    trinfo = TR_info(trcode)
    rqname = trinfo['Name']
    inputs = {k:v for k, v in zip(trinfo['Input'], param)}
    
    print(f'Requesting data for request name: {rqname}')
    
    for event in entrypoint.TransactionCall(rqname, trcode, screenno, inputs):
        print("Got request")
        
        single_names = event.single_data.names
        single_values = event.single_data.values
        output['single'] = {k:v for k, v in zip(single_names, single_values)}
        
        multi_records = [v.values for v in event.multi_data.values]
        multi_df_partial = pd.DataFrame.from_records(multi_records, columns=trinfo['Multi'])
        output['multi'].append(multi_df_partial)
    
    output['multi'] = pd.concat(output['multi'], axis=0).reset_index(drop=True)
    return output

In [65]:
def TR_info(trcode:str, prt:bool=False):
    trinfo = KiwoomOpenApiPlusTrInfo.get_trinfo_by_code(trcode)
    output = {}
    output["Name"] = trinfo.inputs_name
    output["Input"] = list(x.name for x in trinfo.inputs)
    output["Single"] = list(x.name for x in trinfo.single_outputs)
    output["Multi"] = list(x.name for x in trinfo.multi_outputs)
    if prt: print(output)
    return output

In [91]:
tr_example = call_TR('opt10001', ['200710'])

Requesting data for request name: 주식기본정보요청
Got request


In [93]:
tr_example

{'single': {'종목코드': '200710',
  '종목명': '에이디테크놀로지',
  '결산월': '12',
  '액면가': '500',
  '자본금': '70',
  '상장주식': '13427',
  '신용비율': '+6.20',
  '연중최고': '+33500',
  '연중최저': '-25250',
  '시가총액': '3968',
  '시가총액비중': '',
  '외인소진률': '+3.83',
  '대용가': '21880',
  'PER': '66.45',
  'EPS': '445',
  'ROE': '3.6',
  'PBR': '2.32',
  'EV': '42.52',
  'BPS': '12752',
  '매출액': '1642',
  '영업이익': '44',
  '당기순이익': '56',
  '250최고': '+33650',
  '250최저': '-15310',
  '시가': '-30350',
  '고가': '+31000',
  '저가': '-29350',
  '상한가': '+39500',
  '하한가': '-21300',
  '기준가': '30400',
  '예상체결가': '-0',
  '예상체결수량': '0',
  '250최고가일': '20231226',
  '250최고가대비율': '-12.18',
  '250최저가일': '20230210',
  '250최저가대비율': '+93.01',
  '현재가': '-29550',
  '대비기호': '5',
  '전일대비': '-850',
  '등락율': '-2.80',
  '거래량': '371981',
  '거래대비': '+116.51',
  '액면가단위': '원',
  '유통주식': '10650',
  '유통비율': '79.3'},
 'multi': Empty DataFrame
 Columns: []
 Index: []}

In [90]:
df.iloc[1000:1005]

,종목코드,현재가,거래량,거래대금,일자,시가,고가,저가,수정주가구분,수정비율,대업종구분,소업종구분,종목정보,수정주가이벤트,전일종가
1000,,26874,1300809,35643,20200130,27929,28800,26094,,,,,,,
1001,,28617,683119,19805,20200129,29947,30268,28525,,,,,,,
1002,,29442,974791,27431,20200128,26782,29717,26691,,,,,,,
1003,,28617,603228,17136,20200123,28020,29213,27699,,,,,,,
1004,,28433,1251508,33670,20200122,26003,28433,24994,,,,,,,
